In [119]:
import pandas as pd
import numpy as np
import plotly.express as px


In [120]:
df = pd.read_parquet("/home/jpt/gits/iccps_AE/ICCPS2024_AE_Stationing/data_analysis/data/best_solutions.parquet")

In [121]:
searched_dates = df.date.tolist()

In [122]:
import os

def get_folder_names(directory_path):
    folder_names = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
    return folder_names

# Specify the path to the directory
directory_path = 'code_root/scenarios/REAL_WORLD'

# Get all folder names in the specified directory
data_dates = get_folder_names(directory_path)


intersection_set = set(data_dates).intersection(searched_dates)
intersection_list = list(intersection_set)

print("Intersection using set intersection:", intersection_list)


# # Print the folder names
# print("Folder Names:")
# for folder in intersection_list:
#     print(folder)

Intersection using set intersection: ['20210202', '20210928', '20211005', '20201123', '20200522', '20210901', '20200709', '20210211', '20210606', '20210425', '20200714', '20200611', '20201019', '20201017', '20201013', '20210826', '20200307', '20200624', '20200519', '20201023', '20200826', '20200330', '20200613', '20200722', '20200822', '20210401', '20210312', '20201029', '20201205', '20211007', '20210624', '20210302', '20200921', '20210725', '20211014', '20201015', '20210829']


# Plotting

## Figure 4 bottom

In [123]:
plots_dir = "/home/jpt/gits/iccps_AE/ICCPS2024_AE_Stationing/code_root/experiments/TEST/plots"
stops_path = f"{plots_dir}/all_stops.csv"
buses_path = f"{plots_dir}/all_buses.csv"

In [124]:
sdf = pd.read_csv(stops_path, index_col=0)
bdf = pd.read_csv(buses_path, index_col=0)

In [125]:
def parse_filename(df):
    df[['date', 'method', 'depot', 'real', 'test', 'noise', 'iter']] = df['filename'].str.split('_', expand=True)
    return df

In [126]:
# parse filename
sdf = parse_filename(sdf)
bdf = parse_filename(bdf)
sdf.head(1)

,stop_id,total_passenger_ons,total_passenger_offs,total_passenger_walk_away,filename,date,method,depot,real,test,noise
1,GXIRIVEF,21.0,0.0,0.0,20210425_baseline_garage_TEST_TEST_0,20210425,baseline,garage,TEST,TEST,0


In [127]:
bdf = bdf.groupby("filename").agg({"total_served":"sum", "service_kms":"sum", "total_deadsecs":"sum"})
bdf = parse_filename(bdf.reset_index())

# drop nosub for this plot
bdf = bdf[bdf['method'] != "nosub"]

bdf = bdf.groupby("depot").agg({"service_kms":list}).reset_index()
bdf["std"] = bdf["service_kms"].apply(lambda x: np.std(x))
bdf["mean"] = bdf["service_kms"].apply(lambda x: np.mean(x))
bdf["count"] = bdf["service_kms"].apply(len)
bdf["std_e"] = bdf["std"] / bdf["count"].apply(np.sqrt)

In [128]:
bdf

,depot,service_kms,std,mean,count,std_e
0,agency,[153.05],0.0,153.05,1,0.0
1,garage,[153.87],0.0,153.87,1,0.0
2,mcts,[70.99000000000001],0.0,70.99,1,0.0
3,search,[155.97],0.0,155.97,1,0.0


In [129]:

# df.loc[1, "method"] = "GARAGE"
sort_dict = {"garage": 0, "agency": 1, "search": 2, "MCTS": 3}
bdf = bdf.iloc[bdf["depot"].map(sort_dict).sort_values().index]
bdf.depot = bdf['depot'].str.upper()

fig = px.line(bdf, x="depot", y="mean", error_y="std_e", markers=True, title="Line Plot with Error Bars")
fig.update_traces(line={"width": 5})
fig.data[0].error_y.thickness = 5
fig.update_xaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_yaxes(showgrid=True, gridwidth=0.2, gridcolor="lightgrey")
fig.update_layout(
    width=800,
    height=500,
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent paper background
    margin=dict(l=5, r=5, t=5, b=5),
    title="",
    font=dict(
        # family="Courier New, monospace",
        size=18,
        color="black",
    ),
)

fig.update_xaxes(showline=True, linewidth=1, linecolor="black", mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor="black", mirror=True)  # Change x and y-axis labels
fig.update_xaxes(title_text="Stationing Plan")
fig.update_yaxes(title_text="Deadhead kilometers")
# fig.update_xaxes(categoryorder='array', categoryarray= ['GARAGE', 'AGENCY', 'SEARCH', 'MCTS', 'MCTS_5'])
# fig.write_image("./plots/new_deadhead.pdf")
fig.show()

In [ ]:
## Figure 6, need to test on REAL=REAL and TEST=TEST worlds

## Figure 4 Top

In [131]:
df = sdf.groupby("filename").agg({"total_passenger_ons":"sum", "total_passenger_offs":"sum", "total_passenger_walk_away":"sum"})
df = parse_filename(df.reset_index())

# drop nosub for this plot
df = df[df['method'] != "nosub"]

df = df.groupby("depot").agg({"total_passenger_ons":list}).reset_index()
df["std"] = df["total_passenger_ons"].apply(lambda x: np.std(x))
df["mean"] = df["total_passenger_ons"].apply(lambda x: np.mean(x))
df["count"] = df["total_passenger_ons"].apply(len)
df["std_e"] = df["std"] / df["count"].apply(np.sqrt)
df

,depot,total_passenger_ons,std,mean,count,std_e
0,agency,[2633.0],0.0,2633.0,1,0.0
1,garage,[2633.0],0.0,2633.0,1,0.0
2,mcts,[2633.0],0.0,2633.0,1,0.0
3,search,[2633.0],0.0,2633.0,1,0.0


## Figure 5